# Import Packages and Data 

* **Importing packages for preprocssing, visualization and modeling **

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Import Packages for Modeling and Performance Evaluation
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Data Visualization Packages
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from collections import Counter

#Import Packages for Feature Selection
##Univariate feature selection
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2

#Import package for encoding categorical features 
import category_encoders as ce

# Import package calculate accuracy
from sklearn import metrics

#Import package for splitting the dataset
from sklearn.model_selection import train_test_split

* **Importing Training and Test Data**

Since its an old competition, it is not possible to evaluate the prediction using the 'test.csv' dataset provided by amazon. Therefore, this problem is solved by only importing the train.csv file and then spltting the dataset 50/50 for training and testing.

In [ ]:
train = pd.read_csv('../input/train.csv')
X1=train.drop(["ACTION"],axis = 1)
y1=train['ACTION']
train_X, test_X, train_Y, test_Y = train_test_split(X1, y1, test_size=0.5, random_state=42)

# Feature Inspection

In [ ]:
* **Quick Inspection of Train and Test Data**

A quick look at the datasets shows that all the features are categorical and none of them has any missing values.

In [ ]:
train_X.head(5)

In [ ]:
test_X.head(5)

In [ ]:
train_X.info()

In [ ]:
test_X.info()

# Feature Selection

* [Feature selection algorithms](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) help to determine which features significantly influence the output variable  and then eliminate the remaining  features from consideration. This step helps to reduce complexity of a model by reducing the data and enables machine learning algortihms to train faster. 

* The scikit module provides [several ways](https://scikit-learn.org/stable/modules/feature_selection.html) for identifying significant features either by evaluating the statistical correlation with the outcome variable (e.g. univariate feature selection method ) or by measuring  the usefulness of a subset of feature by actually training a machine learning model on it (e.g. recursive feature elimination).

* Considering the size of this particular dataset and the computational time usually required for recursive feature elimination, the universal feature seleciton method is adopted for the first trial. 

* **Applying Univariate Feature Selection Methods to Train data**

In [ ]:
selector = SelectKBest(chi2, k=6)
X_new=selector.fit_transform(train_X, train_Y)
mask = selector.get_support(indices=True)
colname = train_X.columns[mask]
printf('the features to drop are -',colname)

Now that the top 6 significant features have been identified, the remaining features will be dropped from both the training and test dataset. To minimize coding efforts, lets concatenate train and test data to a single dataset prior preprocessing. 

& **Concatenating Train and Test Data into a Single Dataset**

In [ ]:
dataset = pd.concat([train_X, test_X])
dataset.head()

Now, lets drop the features 'RESOURCE', 'ROLE_ROLLUP_1',and 'ROLE_DEPTNAME' from the dataset. 

**Dropping Features From Dataset**

In [ ]:
dataset = dataset.drop(columns=['RESOURCE','ROLE_ROLLUP_2','ROLE_DEPTNAME'],axis=1)
dataset.head(5)

# Feature Encoding

* All the features listed in the dataset are categorical and therefore need to be encoded prior applying any machine learning model to the dataset. Categorical encoding transforms a categorical feature into one or multiple numeric features. [In traditional encoding approaches], this is accomplished by assigning an integar/binary number to each category. For example, one hot coding works by creating a sepearate column for every item (i.e. unique value or type) a categorical variable, and assigns either a '1' or '0' depending on whether the value of the category variable is that item or not.

* However, this feature expansion during encoding can create serious memory problems if the data set has high cardinality features (i.e. categorical variable with large number of items/unique values) as one hot coding creates as many columns as the cardinalities (items/unique values) in the categorical variable. Feature hashing can be useful as an alterntative encoding approach under such circumstances. 

* The core idea behind feature hashing is to map data of arbitrary size to data of fixed size by using a hash function,  instead of maintaining a one-to-one mapping of categorical feature values to new individual columns in the dataset. A detailed discussion of the techqnues can be found [in this link]((https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159)) to get a better understanding of the encodning methods and the benifits of one over the other.

* To decide what feature encoding to apply to process the categorical features for modeling, lets check the number of unique elements in the dataset. 

In [ ]:
**Calculating the Number of Unique Elements in Each Column**

In [ ]:
dataset.nunique()
#dict(Counter(train.RESOURCE))#count the frequancy of each category in RESOURCES

The number of unique values for each feature suggests that this is a dataset with high cardinality features. Therefore, feature hashing was chosen to process the categorical features prior modeling. 

* **Implementing Feature Hashing**

In [ ]:
train_len = len(train_X)
train_X = dataset[:train_len]
test_X = dataset[train_len:]

he = ce.HashingEncoder(cols = ['MGR_ID',"ROLE_CODE","ROLE_FAMILY","ROLE_FAMILY_DESC","ROLE_ROLLUP_1","ROLE_TITLE"], n_components=8)
train_he = he.fit_transform(train_X)
test_he = he.transform(test_X)

# 6. Modeling and Evaluation

* Four different classification models (i.e. DecisionTree Classifier, Random Forest Classifier, Logical Regressor and KNN claissifier) are applied  and then the best model was identified based on the cross validation score.

* For each model, the cross validation was performed by splitting the data (80%/20% for  train/test split), fitting a model and computing the score 5 consecutive times (with different splits each time).

* The average of the  accuracy for the 5 trials were then reported as the CV score for each model.

* **Defining the Input Features for Training and Prediction**

In [ ]:
train_X=train_he
test_X=test_he
train_X.head()

* **Defining Machine Learning Models**

In [ ]:
# Establishing model using train data
DTmodel=DecisionTreeClassifier(random_state=1)
RFmodel=RandomForestClassifier(random_state=1)
logreg = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors = 3) 

**Training the Machine Learning Models**

In [ ]:
#Fitting models
DTmodel.fit(train_X,train_Y)
RFmodel.fit(train_X,train_Y)
logreg.fit(train_X, train_Y)
knn.fit(train_X, train_Y)

**Function to Calculate Cross Validation Score **

In [ ]:
def cross_val(X,x,y):

    scores = cross_val_score(X, x, y, cv=10, scoring = "roc_auc")
    #print("Scores:", scores)
    #print("Mean:", scores.mean())
    #print("Standard Deviation:", scores.std())
    return scores.mean()

**Using the Models for Prediction on Test Data**

In [ ]:
DTpred=DTmodel.predict_proba(test_X).astype(int)
RFpred=RFmodel.predict_proba(test_X).astype(int)
logreg_pred = logreg.predict_proba(test_X).astype(int)
knn_pred = knn.predict_proba(test_X).astype(int)

**Calculate Cross Validation Scores Based on ROC_AUC ** 

In [ ]:
DT_score =round(cross_val(DTmodel,train_X, train_Y) * 100, 2)
RF_score = round(cross_val(RFmodel,train_X, train_Y) * 100, 2)
logreg_score = round(cross_val(logreg,train_X, train_Y) * 100, 2)
knn_score = round(cross_val(knn,train_X, train_Y) * 100, 2)

**Tabulating Scores for Each Model **

In [ ]:
results = pd.DataFrame({'Model': ['DecisionTreeeClaissifier', 'KNN', 'Logistic Regression', 
              'Random ForestClassifier'],'Score': [DT_score , knn_score, logreg_score, 
              RF_score]})
results = results.sort_values(['Score'], ascending=[False])
results.head()

# Compute ROC Curve with Cross Validation


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from scipy import interp
x=test_X
y=test_Y
classifier =[]
classifier.append(RFmodel)
classifier.append(DTmodel)
cv = StratifiedKFold(n_splits=5)
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)

i = 0
for train,test in cv.split(x,y):
    yscore = classifier[0].fit(x.iloc[train],y.iloc[train]).predict_proba(x.iloc[test])
    fpr, tpr, t = roc_curve(y.iloc[test], yscore[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()